In [ ]:
import google.genai as genai
import pandas as pd
import os
from PIL import Image

PROJECT_ID = '$YOUR_PROJECT_ID'
my_api_key = os.getenv('GEMINI_API_KEY')
client = genai.Client(vertexai=False, api_key=my_api_key)

In [ ]:
test_df = pd.read_csv('./sample_data/test-2-4.csv')

In [27]:
prompt = '''
Using given image, please answer question below.
Respond only with left or right.

- Question : '''

In [ ]:
test_df['initial'] = None

for idx in range(len(test_df)):
    now_img_path = test_df.loc[idx, 'image_path']
    now_image = Image.open(now_img_path)
    now_question = test_df.loc[idx, 'question']

    now_prompt = prompt + now_question
    
    response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=[now_prompt, now_image],
        )
    
    test_df.loc[idx, 'initial'] = response.text

    if idx % 10 == 0:
        print(idx)

0
10


In [47]:
optimized_2 = '''Thoroughly examine the provided image to discern all relevant visual details.\nFrom the camera's objective viewpoint, precisely identify the left or right spatial orientation, position, or characteristic that directly answers the question in the query. All determinations of 'left' or 'right' must strictly adhere to the camera's perspective, not any subject's internal or anatomical left/right.\nWhen determining 'left' or 'right' for an object's facing direction, identify the horizontal side of the *image frame* (from the camera's viewpoint) towards which the object's primary face or front is directed. This is *not* the object's anatomical left or right, nor its internal gaze direction. If the query refers to multiple entities or an aggregate, determine the predominant horizontal facing direction as seen by the camera.\nWhen determining 'left' or 'right' for relative positions or features, always use the camera's objective viewpoint. First, identify the specific parts or entities mentioned. If the query asks about a characteristic that implies a vertical comparison (e.g., \"higher\" or \"lower\"), identify the entity or feature that satisfies the vertical condition, then determine its horizontal placement (left or right) *strictly from the camera's perspective*. For all other relative position queries, determine the horizontal placement of the specified parts or features *as they appear from the camera's viewpoint*.\nConfirm your interpretation based solely on the visual evidence.\nRespond only with left or right.\n - query: '''
print(optimized_2)

Thoroughly examine the provided image to discern all relevant visual details.
From the camera's objective viewpoint, precisely identify the left or right spatial orientation, position, or characteristic that directly answers the question in the query. All determinations of 'left' or 'right' must strictly adhere to the camera's perspective, not any subject's internal or anatomical left/right.
When determining 'left' or 'right' for an object's facing direction, identify the horizontal side of the *image frame* (from the camera's viewpoint) towards which the object's primary face or front is directed. This is *not* the object's anatomical left or right, nor its internal gaze direction. If the query refers to multiple entities or an aggregate, determine the predominant horizontal facing direction as seen by the camera.
When determining 'left' or 'right' for relative positions or features, always use the camera's objective viewpoint. First, identify the specific parts or entities mentioned.

In [43]:
test_df['optimized'] = None
for idx in range(len(test_df)):
    now_img_path = test_df.loc[idx, 'image_path']
    now_image = Image.open(now_img_path)
    now_question = test_df.loc[idx, 'question']

    now_prompt = optimized_2 + now_question
    
    response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=[now_prompt, now_image],
        )
    
    test_df.loc[idx, 'optimized'] = response.text

    if idx % 10 == 0:
        print(idx)

0
10


In [49]:
print(test_df[test_df['answer'] == test_df['initial'].str.lower()].shape[0] / len(test_df))
print(test_df[test_df['answer'] == test_df['optimized'].str.lower()].shape[0] / len(test_df))

0.5714285714285714
0.8571428571428571
